***RAG with multiple sources and***
***Agent Tool based custom chatbot***

In [54]:
# Access the environment variables
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
# Load environment variables from the .env file
load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini" ,temperature=0.8)
# Access the environment variables
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")


In [20]:
# Document loader (full directory - only docx)
from langchain_community.document_loaders import DirectoryLoader
path = os.getenv("DATA_PATH")
docs_loader = DirectoryLoader(path, glob="**/*.docx",show_progress=True)
docs = docs_loader.load()


100%|██████████| 11/11 [00:00<00:00, 12.37it/s]


In [21]:
# Document loader (only pdf)
from langchain_community.document_loaders import PyPDFDirectoryLoader
pdf_loader = PyPDFDirectoryLoader(path, glob="**/*.pdf")
pdf_docs = pdf_loader.load()


Overwriting cache for 0 1974


In [22]:
# Combining both pdf & docs
total_docs = docs + pdf_docs
# Splitting into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(total_docs)


In [23]:
## Store into FAISS Vector Database
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
## Vector embedding for text chunks
db1 = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever1 = db1.as_retriever()


In [49]:
# URL loader & Text loader
from langchain_community.document_loaders import TextLoader
url_path = os.getenv('URL_PATH')
text_loader = TextLoader(url_path)
urls = text_loader.load()

from langchain_community.document_loaders import UnstructuredURLLoader
# Split the combined text into individual URLs
url_loader = UnstructuredURLLoader(urls=urls[0].page_content.splitlines())
data = url_loader.load()
# Splitting into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
online_data=text_splitter.split_documents(data)
## FAISS Vector Database
from langchain_community.vectorstores import FAISS
## Vector embedding for text chunks
db2 = FAISS.from_documents(online_data, OpenAIEmbeddings())
retriever2 = db2.as_retriever()


In [50]:
urls[0].page_content.splitlines()


['https://www.ignitiv.com/from-monolith-to-composable-commerce/',
 'https://commercetools.com/blog/from-monolith-to-microservices-migrating-to-a-composable-commerce-architecture-in-only-7-steps',
 'https://alokai.com/blog/from-monolith-to-composable-commerce-the-ultimate-migration-guide',
 'https://fimple.co.uk/how-to-convert-monolithic-architecture-to-microservices-architecture-strangler-pattern/',
 'https://brainhub.eu/library/monolith-to-microservices-using-strangler-pattern',
 'https://dzone.com/articles/monolith-to-microservices-with-the-strangler-patte',
 'https://samirbehara.com/2018/09/10/monolith-to-microservices-using-strangler-pattern/',
 'https://amplication.com/blog/monoliths-to-microservices-using-the-strangler-pattern',
 'https://insights.sei.cmu.edu/blog/8-steps-for-migrating-existing-applications-to-microservices/',
 'https://www.bolt.com/thinkshop/ecommerce-replatforming-migration',
 'https://www.aspiresys.com/composable-commerce-navigating-build/',
 'https://blog.asp

In [51]:
# Create Retriever tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool1 = create_retriever_tool(
    retriever1,
    "doc_search",
    "Search for information from documents, you must use this tool!",
)
retriever_tool2 = create_retriever_tool(
    retriever2,
    "url_search",
    "Search for information from the internet, you must use this tool!",
)


In [52]:
import requests
from flask import Flask, request, jsonify
from langchain_core.messages import HumanMessage
# create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent

app = Flask(__name__)

@app.route('/auras_genai_chatbot/<content>', methods=['GET'])
def query_llm(content):
    prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. You need to use tools for every user's query.Do not answer anything beyond the scope of the two tools.",
        
        ),
        ("placeholder", "{messages}"),
        ("placeholder", "{agent_scratchpad}"),
    ]   
    )
    
    tools = [retriever_tool1, retriever_tool2]

    agent = create_tool_calling_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    response = agent_executor.invoke({"messages": [HumanMessage(content=content)]})
    keys = list(response.keys())
    response_data = response.get(keys[1])
    return jsonify({"response": response_data})
        
if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit




> Entering new AgentExecutor chain...

Invoking: `url_search` with `{'query': 'What is DQAF'}`


Introduction to AURAS: A New Era of Composable Commerce

August 1, 2024 by Shraddha Banerjee Retail auras, Composable commerce, Digital Commerce, ecommerce solutions, MACH architecture 0

Welcome to the future of e-commerce with AURAS, Aspire Systems’ groundbreaking solution designed to revolutionize online commerce. AURAS, or Aspire Unified Reference Architecture Solution, is a custom solution accelerator tailored for composable commerce. This innovative framework accelerates the creation of custom commerce systems by connecting all your business systems through a global integration layer and offering prebuilt templates for faster implementation.

Built on Aspire’s robust Digital Quality Automation Framework (DQAF), AURAS ensures seamless and reliable operations through automated testing. It transforms how enterprises manage e-commerce by adopting a “best-of-breed” approach, allowing bus

127.0.0.1 - - [09/Sep/2024 14:32:57] "GET /auras_genai_chatbot/content=What%20is%20DQAF HTTP/1.1" 200 -


DQAF stands for Digital Quality Automation Framework. It is an advanced, AI-led testing framework specifically designed to address both business and functional challenges in developing retail solutions. Below are some key aspects of DQAF:

1. **Purpose**: DQAF maximizes efficiency, collaboration, and resource utilization for project teams, ensuring that the testing process is accurate and suited to retail operations. 

2. **Flexibility**: It is a tailor-made, flexible, secure, and versatile testing framework that encompasses the entire test data collection process, from custom test report generation to integration with various tools.

3. **Integration**: DQAF allows effective integration of third-party tools, enabling customization up to the last part of the code for end users. It supports automation testing for various eCommerce tools and test management applications.

4. **Advantages**:
   - **Faster Time to Market**: It can reduce regression cycle time by up to 90%, significantly im

...

The architecture of AURAS (Aspire Unified Reference Architecture Solution) comprises several key components and layers:

### High Level Architecture
AURAS architecture is structured into four main layers:
1. **Channels**
2. **Security Layer**
3. **Commerce Layer**
4. **Enterprise Layer**

### Components of AURAS Framework
The AURAS Framework includes three major components:
1. **Commerce Framework**
   - Core Commerce Engine
   - Enterprise & Other Commerce PBCs (Pre-Built Components)
   - API Orchestration & Enterprise Integration Layers
   - FEaaS (Frontend as a Service)
   
2. **Operations**
   - The document does not detail this component in the provided text.

### Overview
AURAS is a custom composable commerce solution accelerator that connects various systems through a global integration layer and includes prebuilt templates to facilitate custom implementations. It is based on the MACH principles (Microservices Architecture, API-first, Cloud-native, and Headless), enabling busine

127.0.0.1 - - [09/Sep/2024 14:35:53] "GET /auras_genai_chatbot/content=What%20is%20the%20architecture%20of%20AURAS HTTP/1.1" 200 -


The architecture of AURAS (Aspire Unified Reference Architecture Solution) consists of several key components and layers:

### High Level Architecture
AURAS architecture includes four primary layers:
1. **Channels**
2. **Security Layer**
3. **Commerce Layer**
4. **Enterprise Layer**

### Components of AURAS Framework
The AURAS Framework is composed of three major components:
1. **Commerce Framework**
   - Core Commerce Engine
   - Enterprise & Other Commerce PBCs (Packaged Business Capabilities)
   - API Orchestration & Enterprise Integration Layers
   - FEaaS (Frontend as a Service)
2. **Operations**

AURAS is designed as a custom composable commerce solution accelerator, facilitating the integration of various systems through a global integration layer and supporting prebuilt PBC templates to expedite custom implementations. It follows the MACH principles (Microservices Architecture, API-first, Cloud-native, and Headless).

> Finished chain.


> Entering new AgentExecutor chain...

I

127.0.0.1 - - [09/Sep/2024 14:36:00] "GET /auras_genai_chatbot/content=What%20is%20the%20architecture%20of%20AURAS HTTP/1.1" 200 -


The architecture of AURAS (Aspire Unified Reference Architecture Solution) consists of several key components and layers. Here's an overview:

### High-Level Architecture
AURAS is structured into four main layers:
1. **Channels**
2. **Security Layer**
3. **Commerce Layer**
4. **Enterprise Layer**

### Components of AURAS Framework
The AURAS Framework is made up of three major components:
1. **Commerce Framework**
   - Core Commerce Engine
   - Enterprise & Other Commerce Pre-Built Components (PBCs)
   - API Orchestration & Enterprise Integration Layers
   - Frontend as a Service (FEaaS)
2. **Operations**
3. **Others** (though not detailed in the provided information)

AURAS is designed to help retailers build custom composable commerce frameworks, facilitating integration across systems through a global integration layer. It emphasizes a "best-of-breed" approach, allowing businesses to select and integrate various commerce services tailored to their specific needs.

> Finished chain.
